In [1]:
import os
import numpy as np
import cv2
import matplotlib.pylab as plt
from sklearn.cluster import KMeans

In [2]:
image_path     = '../../../dataset/images'
training_path  = os.path.join(image_path, 'training')
original_path  = os.path.join(training_path, 'original')
processed_path = os.path.join(training_path, 'processed')

In [3]:
original  = [os.path.join(original_path, image) for image in os.listdir(original_path)]
processed = [os.path.join(processed_path, image) for image in os.listdir(processed_path)]

In [36]:
minsize  = None
minshape = None
for image in original:
    data = cv2.imread(image)
    shape = data.shape
    size = shape[0]*shape[1]

    if (minsize == None)or(size < minsize):
        minshape = shape
        minsize  = size

In [37]:
for image in original:
    data = cv2.imread(image)
    resized = cv2.resize(data, minshape[:2], interpolation = cv2.INTER_CUBIC)
    cv2.imwrite(os.path.join(processed_path, os.path.basename(image)), resized)

In [ ]:
from skimage.restoration import estimate_sigma
from sklearn.cluster import KMeans
import json

# Definir el rango de color naranja en HSV
lower_green = np.array([28, 40, 40])
upper_green = np.array([100, 255, 255])

lower_red_1 = np.array([0, 120, 100])
upper_red_1 = np.array([5, 255, 255])

lower_red_2 = np.array([170, 120, 100])
upper_red_2 = np.array([179, 255, 255])

lower_yellow = np.array([18, 50, 80])
upper_yellow = np.array([33, 255, 255])

lower_orange = np.array([5, 120, 90])
upper_orange = np.array([20, 255, 255])

with open('complicated.json', 'r') as file:
    complicated = json.load(file)

for k, file in enumerate(complicated[9:]):
    if k > 0:
        break
    # Leer la imagen
    image = cv2.imread(file)

    # Convertir la imagen a escala de grises
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    sigma_est  = estimate_sigma(gray_image)

    # Aplicar filtro Non-Local Means para reducir el ruido
    image = cv2.fastNlMeansDenoising(image, None, h=2*sigma_est, templateWindowSize=15, searchWindowSize=30)
    height, width, _ = image.shape
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, mask_gray = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    mask_gray = np.uint8(mask_gray)
 
    # Crear un vector para almacenar los datos de cada píxel
    data_vector = np.zeros((height * width, 10))  # 3 para RGB, 3 para HSV, 3 para LAB, 1 para escala de grises

    # Rellenar el vector con los valores de cada píxel
    for i in range(height):
        for j in range(width):
            pixel = image[i, j]

            # RGB
            data_vector[i * width + j, :3] = pixel

            # HSV
            hsv_pixel = cv2.cvtColor(np.array([[pixel]], dtype=np.uint8), cv2.COLOR_RGB2HSV)[0, 0]
            data_vector[i * width + j, 3:6] = hsv_pixel

            # LAB
            lab_pixel = cv2.cvtColor(np.array([[pixel]], dtype=np.uint8), cv2.COLOR_RGB2LAB)[0, 0]
            data_vector[i * width + j, 6:9] = lab_pixel

            # Escala de grises
            data_vector[i * width + j, 9] = cv2.cvtColor(np.array([[pixel]], dtype=np.uint8), cv2.COLOR_RGB2GRAY)[0, 0]
    
    kmeans = KMeans(n_clusters=2)  # Puedes ajustar el número de clusters según tus necesidades
    kmeans.fit(data_vector)

    # Obtener las etiquetas de los clusters
    labels = kmeans.labels_

    # Crear una máscara basada en las etiquetas
    mask_kmeans = labels.reshape(height, width)
    mask_kmeans = mask_kmeans.astype(np.uint8) * 255

    # Convertir la imagen de BGR a HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv_image)
    l, a, b = cv2.split(cv2.cvtColor(image, cv2.COLOR_BGR2LAB))
    _, mask_h = cv2.threshold(h, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, mask_s = cv2.threshold(s, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, mask_v = cv2.threshold(v, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, mask_l = cv2.threshold(l, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, mask_b = cv2.threshold(b, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, mask_a = cv2.threshold(a, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)


    mask_red    = cv2.inRange(hsv_image, lower_red_1, upper_red_1) | cv2.inRange(hsv_image, lower_red_2, upper_red_2)
    mask_orange = cv2.inRange(hsv_image, lower_orange, upper_orange)
    mask_green  = cv2.inRange(hsv_image, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

    # Unir las máscaras
    combined_mask = mask_red | mask_orange | mask_green | mask_yellow

    #contornos
    #color
    sobel_x = cv2.Sobel(combined_mask, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(combined_mask, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, colorWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #kmeans
    sobel_x = cv2.Sobel(mask_kmeans, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_kmeans, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, kmeansWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #gray
    sobel_x = cv2.Sobel(mask_gray, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_gray, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, grayWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #h
    sobel_x = cv2.Sobel(mask_h, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_h, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, hWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #s
    sobel_x = cv2.Sobel(mask_s, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_s, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, sWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #v
    sobel_x = cv2.Sobel(mask_v, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_v, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, vWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #l
    sobel_x = cv2.Sobel(mask_l, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_l, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, lWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #b
    sobel_x = cv2.Sobel(mask_b, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_b, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, bWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #a
    sobel_x = cv2.Sobel(mask_a, cv2.CV_64F, 1, 0, ksize=9)
    sobel_y = cv2.Sobel(mask_a, cv2.CV_64F, 0, 1, ksize=9)

    # Calcular la magnitud del gradiente
    magnitude = np.sqrt(sobel_x**2 + sobel_y**2)
    magnitude = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    magnitude = np.uint8(magnitude)

    # Binarizar la imagen con Otsu
    _, aWcnt = cv2.threshold(magnitude, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    #Encontrar contornos
    kmeans_cnt, _ = cv2.findContours(kmeansWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    color_cnt, _ = cv2.findContours(colorWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    gray_cnt, _ = cv2.findContours(grayWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    h_cnt, _ = cv2.findContours(hWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    s_cnt, _ = cv2.findContours(sWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    v_cnt, _ = cv2.findContours(vWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    l_cnt, _ = cv2.findContours(lWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    b_cnt, _ = cv2.findContours(bWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    a_cnt, _ = cv2.findContours(aWcnt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    #Dibujar contornos
    #templates
    tcolor = np.zeros((height, width), dtype=np.uint8)
    tkmeans = np.zeros((height, width), dtype=np.uint8)
    tgray = np.zeros((height, width), dtype=np.uint8)
    th = np.zeros((height, width), dtype=np.uint8)
    ts = np.zeros((height, width), dtype=np.uint8)
    tv = np.zeros((height, width), dtype=np.uint8)
    tl = np.zeros((height, width), dtype=np.uint8)
    tb = np.zeros((height, width), dtype=np.uint8)
    ta = np.zeros((height, width), dtype=np.uint8)

    #dibujar
    cv2.drawContours(tcolor, color_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(tkmeans, kmeans_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(tgray, gray_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(th, h_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(ts, s_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(tv, v_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(tl, l_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(tb, b_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(ta, a_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno

    #superposición
    whole = np.zeros((height, width), dtype=np.uint8)
    cv2.drawContours(whole, color_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, kmeans_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, gray_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, h_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, s_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, v_cnt, -1,255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, l_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, b_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno
    cv2.drawContours(whole, a_cnt, -1, 255, 1)  # El último argumento es el grosor del contorno

    #plotting

    plt.figure(figsize =(10,5))
    plt.subplot(1, 10, 1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(os.path.basename(file))
    plt.subplot(1, 10, 2)
    plt.imshow(combined_mask, cmap='gray')
    plt.title('color')
    plt.subplot(1, 10, 3)
    plt.imshow(mask_kmeans, cmap='gray')
    plt.title('kmeans')
    plt.subplot(1, 10, 4)
    plt.imshow(mask_gray, cmap='gray')
    plt.title('gray_umbral')
    plt.subplot(1, 10, 5)
    plt.imshow(mask_h, cmap='gray')
    plt.title('h')
    plt.subplot(1, 10, 6)
    plt.imshow(mask_s, cmap='gray')
    plt.title('s')
    plt.subplot(1, 10, 7)
    plt.imshow(mask_v, cmap='gray')
    plt.title('v')
    plt.subplot(1, 10, 8)
    plt.imshow(mask_l, cmap='gray')
    plt.title('l')
    plt.subplot(1, 10, 9)
    plt.imshow(mask_b, cmap='gray')
    plt.title('b')
    plt.subplot(1, 10, 10)
    plt.imshow(mask_a, cmap='gray')
    plt.title('a')
    plt.show()

    plt.figure(figsize =(10,5))
    plt.subplot(1, 10, 1)
    plt.imshow(tcolor, cmap='gray')
    plt.title('color')
    plt.subplot(1, 10, 2)
    plt.imshow(tkmeans, cmap='gray')
    plt.title('kmeans')
    plt.subplot(1, 10, 3)
    plt.imshow(tgray, cmap='gray')
    plt.title('gray_umbral')
    plt.subplot(1, 10, 4)
    plt.imshow(th, cmap='gray')
    plt.title('h')
    plt.subplot(1, 10, 5)
    plt.imshow(ts, cmap='gray')
    plt.title('s')
    plt.subplot(1, 10, 6)
    plt.imshow(tv, cmap='gray')
    plt.title('v')
    plt.subplot(1, 10, 7)
    plt.imshow(tl, cmap='gray')
    plt.title('l')
    plt.subplot(1, 10, 8)
    plt.imshow(tb, cmap='gray')
    plt.title('b')
    plt.subplot(1, 10, 9)
    plt.imshow(ta, cmap='gray')
    plt.title('a')
    plt.subplot(1, 10, 10)
    plt.imshow(whole, cmap='gray')
    plt.title('whole')
    plt.show()